# Observed Air Quality (PurpleAir) - REAL TIME

This notebook retrieves readings from PurpleAir Sensors in Minneapolis and cleans the entries and saves the results as a csv file.

Documentation is available here: https://api.purpleair.com.
You can read this article for help getting started: https://community.purpleair.com/t/making-api-calls-with-the-purpleair-api/180.

From PurpleAir: 

"The data from individual sensors will update no less than every 30 seconds. As a courtesy, we ask that you limit the number of requests to no more than once every 1 to 10 minutes, assuming you are only using the API to obtain data from sensors. If retrieving data from multiple sensors at once, please send a single request rather than individual requests in succession.

The PurpleAir historical API is released as of July 18, 2022. For more information, view this post: https://community.purpleair.com/t/new-version-of-the-purpleair-api-on-july-18th/1251.

Please let us know if you have any questions or concerns, and have a great day!"

A paper on this process: https://doi.org/10.5194/amt-14-4617-2021 (Link for [Download](https://www.researchgate.net/publication/352663348_Development_and_application_of_a_United_States-wide_correction_for_PM25_data_collected_with_the_PurpleAir_sensor) )

Chat on which PM Estimate to use: https://community.purpleair.com/t/pm2-5-algorithms/3972/6

In [1]:
### Import Packages

# File manipulation

import os # For working with Operating System
import requests # Accessing the Web
import datetime as dt # Working with dates/times
import io # Input/Output Bytes objects
import time # For sleep in for loop

# Analysis

import numpy as np
import pandas as pd

## Definitions

In [2]:
spike_threshold = 28 # Micgrograms per meter cubed

In [3]:
# This is my personal API key... Please use responsibly!

api = input('Please enter your Purple Air api key')

Please enter your Purple Air api key 51592903-B445-11ED-B6F4-42010A800007


### Sensor Indices

In [4]:
# Sensor Indices (from City of Minneapolis)

indices_path = os.path.join('..', 'Historic_PurpleAir', 'PA IDs and indexes.xlsx')

sensor_info = pd.read_excel(indices_path) # Load as DataFrame

sensor_ids = sensor_info['Sensor Index'].dropna().astype(int) # This should be an iterable of the sensor ids as integers

In [5]:
def getSensorsData(query='', api_read_key=''):

    # my_url is assigned the URL we are going to send our request to.
    url = 'https://api.purpleair.com/v1/sensors?' + query
    
    # print('Here is the full url for the API call:\n\n', url)

    # my_headers is assigned the context of our request we want to make. In this case
    # we will pass through our API read key using the variable created above.
    my_headers = {'X-API-Key':api_read_key}

    # This line creates and sends the request and then assigns its response to the
    # variable, r.
    response = requests.get(url, headers=my_headers)

    # We then return the response we received.
    return response

## Actual RealTime

In [6]:
# Actual RealTime

sensor_string = 'show_only=' + '%2C'.join(sensor_ids.astype(str))

query = 'fields=pm2.5_cf_1&' + sensor_string

response = getSensorsData(query, api)

response_dict = response.json() # Read response as a json (dictionary)

col_names = response_dict['fields']
data = np.array(response_dict['data'])

sensors_df = pd.DataFrame(data, columns = col_names)

In [7]:
sensors_df.head()

,sensor_index,pm2.5_cf_1
0,142718.0,1.2
1,142720.0,2.9
2,142726.0,1.0
3,142724.0,0.9
4,142730.0,1.1


## For Summary of Previous Day

### Summary Statistics Functions

In [21]:
%run ../Historic_PurpleAir/Summary_Functions.py

print('Stat Names:\n\n', summary_stats_names, '\n')
print('Stat Types:\n\n',summary_stats_dtypes, '\n')

print('Function Names:\n\n', summary_stats_functions)

Stat Names:

 ['n_observations', 'humidity_fullDay_mean', 'temperature_fullDay_mean', 'pressure_fullDay_mean', 'pm25_fullDay_mean', 'pm25_fullDay_min', 'pm25_fullDay_minTime', 'pm25_fullDay_max', 'pm25_fullDay_maxTime', 'pm25_fullDay_std', 'pm25_fullDay_minutesAbove12ug', 'pm25_morningRush_mean', 'pm25_morningRush_min', 'pm25_morningRush_minTime', 'pm25_morningRush_max', 'pm25_morningRush_maxTime', 'pm25_morningRush_std', 'pm25_eveningRush_mean', 'pm25_eveningRush_min', 'pm25_eveningRush_minTime', 'pm25_eveningRush_max', 'pm25_eveningRush_maxTime', 'pm25_eveningRush_std', 'pm25_daytimeAmbient_mean', 'pm25_daytimeAmbient_min', 'pm25_daytimeAmbient_minTime', 'pm25_daytimeAmbient_max', 'pm25_daytimeAmbient_maxTime', 'pm25_daytimeAmbient_std', 'pm25_nighttimeAmbient_mean', 'pm25_nighttimeAmbient_min', 'pm25_nighttimeAmbient_minTime', 'pm25_nighttimeAmbient_max', 'pm25_nighttimeAmbient_maxTime', 'pm25_nighttimeAmbient_std'] 

Stat Types:

 [<class 'int'>, <class 'float'>, <class 'float'>, <

### Functions

In [9]:
# QAQC

def qaqc(df):
    '''This function wil perform some basic QAQC
    '''
    
    clean_df = df.copy()
    
    # Convert timestamp to datetime
    
    clean_df['timestamp'] = pd.to_datetime(clean_df['timestamp'], unit='s')
    
    # Remove obvious error values
    
    clean_df = clean_df[clean_df.pm25 < 1000] 
    
    # Remove NaNs
    
    clean_df = clean_df.dropna()
    
    return clean_df

# Remove and record Spikes

def get_spikes(df, spike_threshold):
    '''This function removes spikes from a dataframe 
    and returns both the new dataframe
    and a separate spike dataframe
    '''
    
    df_w_spikes = df.copy()
    
    condition = (df.pm25 > spike_threshold)
    
    df_w_spikes['is_spike'] = condition
    
    spikes = df_w_spikes[condition].copy()
    
    return df_w_spikes, spikes

# Get Summary Stats

def get_summary_stats(df):
    ''' This is the main function. It will run all of our functions that get summary stats
    and return as a list.
    '''
    
    stats = []
    
    # Run the functions
    
    for f in summary_stats_functions:
        stats += f(df)
    
    return stats

### Set Up Parameters for Query

In [10]:
### Query Strings

# Average string (in minutes) 1440 is 1 day average

avg_string = 'average=10'

# Environmental fields

env_fields = ['humidity', 'temperature', 'pressure','pm2.5_cf_1']

env_fields_string = 'fields=' + '%2C%20'.join(env_fields)

# My Header

my_headers = {'X-API-Key': api}

## The Loop

In [12]:
# Dates

today = dt.datetime.combine(dt.datetime.today(), dt.datetime.min.time()) # set to today @ midnight 
yesterday = today - dt.timedelta(days=1)

# Set up Timestamp for query    

start_timestamp = int(yesterday.timestamp())
end_timestamp = int(today.timestamp())

time_string = 'start_timestamp=' + str(start_timestamp) + '&end_timestamp=' + str(end_timestamp)

print('Last Run on ', today)

Last Run on  2023-05-01 00:00:00


In [13]:
## Iterables

sensor_ids = sensors_df.sensor_index.astype(int)

In [22]:
## Initialize Storage

# Daily Summary

cols = ['sensor_index', 'date'] + summary_stats_names

datatypes = [int, str] + summary_stats_dtypes

dtypes = np.dtype(list(zip(cols, datatypes)))

daily_summary_df = pd.DataFrame(np.empty(0, dtype = dtypes))

# Daily Summary (No Spikes)

cols = ['sensor_index', 'date'] + summary_stats_names

datatypes = [int, str] + summary_stats_dtypes

dtypes = np.dtype(list(zip(cols, datatypes)))

daily_summary_no_spikes_df = pd.DataFrame(np.empty(0, dtype = dtypes))

# Spikes

all_spikes_df = pd.DataFrame(np.empty(0, dtype = [('sensor_index', int),
                                                  ('timestamp', pd._libs.tslibs.timestamps.Timestamp),
                                                  ('pm25', float)]
                                 )
                        )

# No Data for sensor

no_data = pd.DataFrame(np.empty(0, dtype = [('sensor_index', int),
                                            ('date', str)
                                           ]))

In [23]:
   
# Iterate through the Sensors

for sensor_id in sensor_ids:

    # For skipping to last spot

    is_done = (daily_summary_df.sensor_index == int(sensor_id))
    is_no_data = (no_data.sensor_index == int(sensor_id))

    # If either of these has a true, it has been parsed
    # True is not in both, then we should process
    if (True not in is_done.values) & (True not in is_no_data.values):

        ### Actual Loop

        time.sleep(3)

        # Base URL
        base_url = f'https://api.purpleair.com/v1/sensors/{sensor_id}/history/csv?'

        # Put it all together
        query_url = base_url + '&'.join([time_string, avg_string, env_fields_string])

        response = requests.get(query_url, headers=my_headers)

        if response.status_code == 200:

            # Read response as CSV data
            csv_data = response.content.decode('utf-8')

            if csv_data.count('\n') == 1: # There is only one line (empty data)
                # print(f"No data for sensor {sensor_id} on {datelist[i]}")

                no_data.loc[len(no_data.index)] = [sensor_id, yesterday.date()]

            else:
                # Parse CSV data into pandas DataFrame
                df_individual_sensor = pd.read_csv(io.StringIO(csv_data),
                                                   header=0
                                                  )[['time_stamp', 'humidity', 'temperature', 
                                                     'pressure', 'pm2.5_cf_1']]

                df_individual_sensor.columns = ['timestamp', 'humidity', 'temperature', 
                                                'pressure', 'pm25']

                # Perform QAQC

                clean = qaqc(df_individual_sensor)

                # Remove Spikes & Concatenate to main storage of spikes

                clean_w_spikes, spikes = get_spikes(clean, spike_threshold)

                spikes['sensor_index'] = int(sensor_id)

                all_spikes_df = pd.concat([all_spikes_df, 
                                           spikes[['sensor_index',
                                                    'timestamp',
                                                    'pm25']]
                                          ],
                                           ignore_index=True)

                # Get Stats (With Spikes)

                sum_stats = get_summary_stats(clean_w_spikes)

                # Add to the daily summary dataframe

                row = [int(sensor_id), str(yesterday.date())] + sum_stats

                daily_summary_df.loc[len(daily_summary_df.index)] = row

                # Get Stats (Without Spikes)

                no_spikes = clean_w_spikes[clean_w_spikes.is_spike == False]

                sum_stats = get_summary_stats(no_spikes)

                # Add to the daily summary dataframe

                row = [int(sensor_id), str(yesterday.date())] + sum_stats

                daily_summary_no_spikes_df.loc[len(daily_summary_no_spikes_df.index)] = row

        else:
            print(f"Error fetching data for sensor {sensor_id}: {response.status_code} on {yesterday.date()}")

KeyboardInterrupt: 

In [24]:
daily_summary_df.tail(10)

# Drop Duplicates
# daily_summary_df = daily_summary_df.drop_duplicates(ignore_index = True).copy()

,sensor_index,date,n_observations,humidity_fullDay_mean,temperature_fullDay_mean,pressure_fullDay_mean,pm25_fullDay_mean,pm25_fullDay_min,pm25_fullDay_minTime,pm25_fullDay_max,...,pm25_daytimeAmbient_minTime,pm25_daytimeAmbient_max,pm25_daytimeAmbient_maxTime,pm25_daytimeAmbient_std,pm25_nighttimeAmbient_mean,pm25_nighttimeAmbient_min,pm25_nighttimeAmbient_minTime,pm25_nighttimeAmbient_max,pm25_nighttimeAmbient_maxTime,pm25_nighttimeAmbient_std
0,142718,2023-04-30,144,46.856944,50.058333,976.331542,1.355007,0.0,08:50:00,16.967,...,12:00:00,0.918,13:40:00,0.212792,3.029211,1.101,02:10:00,7.599,00:30:00,2.138295
1,142720,2023-04-30,144,44.252778,49.573611,976.220472,0.655521,0.0,08:50:00,3.577,...,12:00:00,3.384,14:20:00,0.895240,1.020526,0.736,00:20:00,1.352,00:30:00,0.171751
2,142726,2023-04-30,144,47.052660,49.194792,976.060986,1.197368,0.0,09:10:00,6.703,...,14:00:00,6.605,12:10:00,1.922584,1.301421,0.887,02:30:00,2.383,00:30:00,0.404918


In [25]:
daily_summary_no_spikes_df.tail(10)

# To Remove Duplicates
# daily_summary_no_spikes_df = daily_summary_no_spikes_df.drop_duplicates(ignore_index = True).copy()

,sensor_index,date,n_observations,humidity_fullDay_mean,temperature_fullDay_mean,pressure_fullDay_mean,pm25_fullDay_mean,pm25_fullDay_min,pm25_fullDay_minTime,pm25_fullDay_max,...,pm25_daytimeAmbient_minTime,pm25_daytimeAmbient_max,pm25_daytimeAmbient_maxTime,pm25_daytimeAmbient_std,pm25_nighttimeAmbient_mean,pm25_nighttimeAmbient_min,pm25_nighttimeAmbient_minTime,pm25_nighttimeAmbient_max,pm25_nighttimeAmbient_maxTime,pm25_nighttimeAmbient_std
0,142718,2023-04-30,144,46.856944,50.058333,976.331542,1.355007,0.0,08:50:00,16.967,...,12:00:00,0.918,13:40:00,0.212792,3.029211,1.101,02:10:00,7.599,00:30:00,2.138295
1,142720,2023-04-30,144,44.252778,49.573611,976.220472,0.655521,0.0,08:50:00,3.577,...,12:00:00,3.384,14:20:00,0.895240,1.020526,0.736,00:20:00,1.352,00:30:00,0.171751
2,142726,2023-04-30,144,47.052660,49.194792,976.060986,1.197368,0.0,09:10:00,6.703,...,14:00:00,6.605,12:10:00,1.922584,1.301421,0.887,02:30:00,2.383,00:30:00,0.404918


In [26]:
all_spikes_df

# To Remove Duplicates
# all_spikes_df = all_spikes_df.drop_duplicates(ignore_index = True).copy()

,sensor_index,timestamp,pm25


In [27]:
no_data

# To Remove Duplicates
# no_data = no_data.drop_duplicates(ignore_index = True).copy()

,sensor_index,date


In [28]:
daily_summary_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 0 to 2
Data columns (total 37 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   sensor_index                   3 non-null      int64  
 1   date                           3 non-null      object 
 2   n_observations                 3 non-null      int64  
 3   humidity_fullDay_mean          3 non-null      float64
 4   temperature_fullDay_mean       3 non-null      float64
 5   pressure_fullDay_mean          3 non-null      float64
 6   pm25_fullDay_mean              3 non-null      float64
 7   pm25_fullDay_min               3 non-null      float64
 8   pm25_fullDay_minTime           3 non-null      object 
 9   pm25_fullDay_max               3 non-null      float64
 10  pm25_fullDay_maxTime           3 non-null      object 
 11  pm25_fullDay_std               3 non-null      float64
 12  pm25_fullDay_minutesAbove12ug  3 non-null      int64  

In [29]:
# # Save it!?!

# daily_summary_df.to_csv('daily_summaries_today.csv', index = False)

# daily_summary_no_spikes_df.to_csv('daily_summaries_no_spikes_today.csv', index = False)

# all_spikes_df.to_csv('all_spikes_today.csv', index = False)

# no_data.to_csv('no_data_sensors_today.csv', index = False)